In [34]:
import matplotlib.pyplot as plt
from pandas import read_csv
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report,roc_curve, auc, plot_roc_curve
from sklearn.metrics import confusion_matrix

In [35]:
df = pd.read_csv('/root/liver_spleen_seg/radiomics/model/LSPS2.csv')
train = df.loc[df["GROUP"] == 1]
test = df.loc[df["GROUP"] == 2]

In [36]:
#label & model results
train_label=train.iloc[:,6]
test_label=test.iloc[:,6]
train_PRE=train.iloc[:,12]
test_PRE=test.iloc[:,12]

def Find_Optimal_Cutoff(TPR, FPR, threshold):
    y = TPR - FPR
    Youden_index = np.argmax(y)  # Only the first occurrence is returned.
    optimal_threshold = threshold[Youden_index]
    point = [FPR[Youden_index], TPR[Youden_index]]
    return optimal_threshold, point
# Youden_index
fpr1,tpr1,thresholds1 = roc_curve(train_label,train_PRE)
optimal_th, optimal_point = Find_Optimal_Cutoff(TPR=tpr1, FPR=fpr1, 
                                                          threshold=thresholds1)



In [37]:
def Find_All_Cutoff(label,PRE):
    FPR,TPR,threshold = roc_curve(label,PRE)
    COUNT=0
    while COUNT < len(threshold):
        optimal_th=threshold[COUNT]
        TRAIN_PRED_LABEL = np.zeros([len(PRE), 1])
        TRAIN_PRED_LABEL[PRE >= optimal_th] = 1
        tn, fp, fn, tp = confusion_matrix(label, TRAIN_PRED_LABEL).ravel()
        train_sens = tp / (tp + fn)
        train_spec = tn / (tn + fp)
        PPV = tp / (tp + fp)
        NPV = tn / (tn + fn)
        #print(optimal_th,tn, fp, fn, tp,PPV,NPV)
        print(COUNT,'%.3f'%optimal_th,'\t'," tp\t",tp,'\t'," fp\t",fp,'\t'," tn\t",tn,'\t'," fn\t",fn,'\t',"PPV\t",'%.3f'%PPV,'\t',"NPV\t",'%.3f'%NPV,'\t',
          "sensetivity\t",'%.3f'%train_sens,'\t',"specificity\t",'%.3f'%train_spec) 
        COUNT=COUNT+1

In [ ]:
#所有阈值对应的结果就出来了，可以挑选需要的阈值
Find_All_Cutoff(train_label,train_PRE)